In [ ]:
board = Board()
board.play()

In [79]:
board = Board()
assert not board.is_check('white')
assert not board.is_check('black')
assert not board.is_checkmate('white')
assert not board.is_checkmate('black')

boards = [
    Board([
        [King('black'),  Tower('white')],
        [Tower('white'), Queen('white')],
    ]),
    Board([
        [King('black'),  Horse('white'), Empty()],
        [Empty(),        Queen('white'), Empty()],
        [Empty(),        Empty(),        Bishop('white')]
    ]),
    Board([
        [King('black'),  Empty(), Horse('white')],
        [Empty(),        Empty(), Empty()],
        [Horse('white'), Empty(), Bishop('white')]
    ]),
]

for board in boards:
    assert board.is_check('black')
    assert board.is_checkmate('black')
    
    
boards = [
    Board([
        [King('black'),  Empty(),        Horse('white')],
        [Empty(),        Empty(),        Empty()],
        [Empty(),        Horse('white'), Bishop('white')]
    ]),
]

for board in boards:
    assert not board.is_checkmate('black')
    

In [68]:
print(board)

8 ♚ ♖
7 ♖ ♕
  a b c d e f g h


In [87]:
import random

random.choice(list(board.moves('white')))


(2, 1)

In [102]:
from random import choice


class Opponent:
    """The opponent to play against.
    
    Simulates new boards to find the best new board position.
    """
    
    
    def __init__(self, board):
        self.board = board
        self.color = 'black'
        self.piece_value = {
            'Empty': 0,
            'Pawn': 1,
            'Horse': 3,
            'Bishop': 3,
            'Tower': 5,
            'Queen': 9,
            'King': 1e6,
        }
        
    def value(self, board):
        total_value = 0
        for row in board:
            for piece in row:
                pv = self.piece_value[type(piece).__name__]
                if piece.color == self.color:
                    total_value += pv
                else:
                    total_value -= pv
        return total_value
    
    def simulation(self, board):
        board.checkmate = False
        try:
            while not board.checkmate:

                moves = board.moves(board.current_color)
                from_ = random.choice([
                    from_ for from_ in moves if len(moves[from_]) != 0
                ])
                to = random.choice(moves[from_])

                board = board.simulate_move(from_, to)
                board.update()
        except:
            print(board)

In [165]:
board = Board()
opponent = Opponent(board)
opponent.simulation(board)

black lost the game...

8         ♔      
7               ♚
6                
5                
4         ♘      
3             ♕  
2               ♖
1                
  a b c d e f g h


In [88]:
import numpy as np
import copy

from itertools import chain

class Board:    
    """The chess board.
    
    The board contains all the pieces and the pieces contains a reference to the board.
    """
    
    def __init__(self, board = None):
        if board is None:
            self.board = self._create_board()
        else:
            self.board = board
        self.current_color = 'white'
    
    @property
    def board(self):
        """Is the board of the correct type etc."""
        return self._board
            
    @board.setter
    def board(self, value):
        
        for row in value:
            for piece in row:
                piece.board = self
        
        if isinstance(value, list):
            value = np.array(value)

        # Run checks
        if not isinstance(value, np.ndarray):
            raise ValueError('Board should be a numpy array.')
            
        self._board = value

    @board.deleter
    def board(self):
        del self._board
    
    def __getitem__(self, position):
        if any(idx < 0 for idx in position):
            return None
        try:
            return self.board[position]
        except IndexError:
            return None
    
    def __setitem__(self, position, piece):
        self.board[position] = piece 

    def __len__(self):
        return 8
    
    def __eq__(self, other):
        if isinstance(other, Piece):
            return self.board == other
        elif isinstance(other, Board):
            return bool(self.board == other.board)
        return False
    
    def __iter__(self):
        return self.board.__iter__()
    
    def __str__(self):
        lines = []
        for idx, row in enumerate(self):
            lines.append(str(8 - idx) + ' ' + ' '.join(map(str, row)))
        lines.append('  a b c d e f g h')
        return '\n'.join(lines)
    
    def play(self):
        self.checkmate = False
        while not self.checkmate:
            print(self)
            print()
            str_from = input('Move from: ')
            str_to = input('Move to: ')
            print()
            try:
                from_ = self.translate(str_from)
                to = self.translate(str_to)

                if isinstance(self[from_], Empty):
                    print("Not a piece...\n")
                    continue
                if self[from_] is None:
                    print("Not on the board...\n")
                    continue
                if to not in self[from_].moves():
                    print("Invalid move...\n")
                    continue
                if self[from_].color != self.current_color:
                    print('Not this players turn...\n')
                    continue

            except:
                print("Invalid move...\n")
                continue
            
            self.move(from_, to)
    
    def update(self):
        self.update_color()
        self.update_checkmate()
            
    def translate(self, chess_notation):
        letter, number = chess_notation
        i = 8 - int(number)
        j = dict(zip(
            ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'],
            range(len(self))
        ))[letter]
        return i, j
            
    def copy(self):
        return copy.deepcopy(self)
            
    def move(self, from_, to):
        self[to] = self[from_]
        self[from_] = Empty(None, self)
        return self

    def _create_board(self):
        """Create a chess board."""
        return np.array([
            self.back_row('white'),
            self.pawn_row('white'),
        ] + [
            self.empty_row() for i in range(4)
        ] + [
            self.pawn_row('black'),
            self.back_row('black'),
        ])
    
    def back_row(self, color):
        return [
            Tower(color, self),
            Horse(color, self),
            Bishop(color, self),
            King(color, self),
            Queen(color, self),
            Bishop(color, self),
            Horse(color, self),
            Tower(color, self),
        ]
    
    def pawn_row(self, color):
        return [Pawn(color, self) for j in range(len(self))]
    
    def empty_row(self):
        return [Empty(None, self) for j in range(len(self))]
    
    def update_color(self):
        self.current_color = self.other_color(self.current_color)
    
    def other_color(self, color):
        from_to = {'white': 'black', 'black': 'white'}
        return from_to[color]

    def moves(self, color):
        moves = {}
        for row in self.board:
            for piece in row:
                if piece.color == color:
                    moves[piece.position] = piece.moves()
        return moves
    
    def get_king(self, color):
        for row in self.board:
            for piece in row:
                if (piece.color == color) & isinstance(piece, King):
                    return piece
        raise ValueError(f'Missing king of color {color}')
    
    def king_moves(self, color):
        return self.get_king(color).moves()
    
    def update_checkmate(self):
        if self.is_checkmate(self.current_color):
            self.checkmate = True
            print(f"{self.current_color} lost the game...\n")
            print(self)
    
    def is_checkmate(self, color):
        return (
            self.is_check(color)
            & self.cant_move_out_of_check(color)
        )

    def is_check(self, color):
        king = self.get_king(color)
        enemy_moves = list(chain(
            *self.moves(self.other_color(color)).values()
        ))
        return king.position in enemy_moves
    
    def cant_move_out_of_check(self, color):
        king = self.get_king(color)
        for to in king.moves():
            new_board = self.simulate_move(king.position, to)
            if not new_board.is_check(color):
                return False
        return True
        
    def simulate_move(self, from_, to):
        new_board = self.copy()
        return new_board.move(from_, to)
        

In [23]:
class Piece:

    def __init__(self, color=None, board=None):
        self.color = color
        self.board = board
    
    @property
    def position(self):
        i, j = np.where(self.board == self)
        return int(i), int(j)
            
    @position.setter
    def position(self, value):
        self.board.move(
            from_=self.position,
            to=value
        )
    
    def move(self, value):
        self.board.move(
            from_=self.position,
            to=value
        )
    
    def moves(self):
        """Should return a list of all moves e.g. [(1,1), (1,2), ..]."""
        raise NotImplementedError()
    
    def __str__(self):
        """Return a string representation of the piece."""
        if self.color is None:
            return ' '
        return getattr(self, self.color)

    def print_moves(self):
        new_board = board.copy()
        for move in self.moves():
            new_board[move] = '.'
        print(new_board)

In [24]:
class Empty(Piece):

    def moves(self):
        return None

In [25]:
class King(Piece):
    
    white = u'♔'
    black = u'♚'

    def moves(self):
        moves = []
        i, j = self.position
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                new_position = (i + di, j + dj) 
                if self.board[new_position] is None:
                    continue
                elif self.board[new_position].color == self.color:
                    continue
                moves.append(new_position)
        return moves

In [26]:
class Pawn(Piece):
    
    white = u'♙'
    black = u'♟'
    
    def moves(self):
        i, j = self.position
        moves = [(i, j)]
        moves.extend(self.forward_move(i, j))
        moves.extend(self.attacking_moves(i, j))
        return moves
    
    def forward_move(self, i, j):
        if self.color == 'white':
            moves = [(i + 1, j)]
            if i == 1:
                moves = [(i + 1, j), (i + 2, j)]
                
        elif self.color == 'black':
            moves = [(i - 1, j)]
            if i == 6:
                moves = [(i - 1, j), (i - 2, j)]
                
        else:
            raise ValueError(f"Invalid color {self.color}")
        
        if all(
            isinstance(self.board[new_position], Empty) 
            for new_position in moves
        ):
            return moves

        return []
        
    def attacking_moves(self, i, j):
        if self.color == 'white':
            moves = [(i + 1, j - 1), (i + 1, j + 1)]
            moves = filter(
                lambda new_position: self.board[new_position] == 'black',
                moves
            )
        elif self.color == 'black':
            moves = [(i - 1, j - 1), (i - 1, j + 1)]
            moves = filter(
                lambda new_position: self.board[new_position] == 'white',
                moves
            )
        else:
            raise ValueError(f"Invalid color {self.color}")
        return moves

In [27]:
class Tower(Piece):
    
    white = u'♖'
    black = u'♜'
        
    def moves(self):
        i, j = self.position
        moves = [(i, j)]
        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            for step in range(1, len(self.board)):
                new_position = (i + step * di, j + step * dj)
                if self.board[new_position] is None:
                    break
                elif isinstance(self.board[new_position], Empty):
                    moves.append(new_position)
                elif self.board[new_position].color == self.color:
                    break
                elif self.board[new_position].color != self.color:
                    moves.append(new_position)
                    break
                

        return moves

In [28]:
class Bishop(Piece):
    
    white = u'♗'
    black = u'♝'
    
    def moves(self):
        i, j = self.position
        moves = [(i, j)]
        for di in [-1, 1]:
            for dj in [-1, 1]:
                for s in range(1, len(self.board)):
                    new_position = (i + s * di, j + s * dj)
                    if self.board[new_position] is None:
                        break
                    elif isinstance(self.board[new_position], Empty):
                        moves.append(new_position)
                    elif self.board[new_position].color == self.color:
                        break
                    elif self.board[new_position].color != self.color:
                        moves.append(new_position)
                        break
        return moves

In [29]:
class Horse(Piece):
    
    white = u'♘'
    black = u'♞'
    
    def moves(self):
        i, j = self.position
        moves = [(i, j)]
        for di, dj in [
            (-2, -1), (2, 1), (2, -1), (-2, 1),
            (-1, -2), (1, 2), (1, -2), (-1, 2),
        ]:
            new_position = (i + di, j + dj)
            if self.board[new_position] is None:
                continue
            elif isinstance(self.board[new_position], Empty):
                moves.append(new_position)
            elif self.board[new_position].color == self.color:
                continue
            elif self.board[new_position].color != self.color:
                moves.append(new_position)
        return moves

In [42]:
class Queen(Piece):
    
    white = u'♕'
    black = u'♛'
    
    def moves(self):
        i, j = self.position
        moves = [(i, j)]
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                for s in range(1, len(self.board)):
                    new_position = (i + s * di, j + s * dj)
                    if self.board[new_position] is None:
                        break
                    elif isinstance(self.board[new_position], Empty):
                        moves.append(new_position)
                    elif self.board[new_position].color == self.color:
                        break
                    elif self.board[new_position].color != self.color:
                        moves.append(new_position)
                        break
        return moves